# 第7章: お問い合わせフォーム - 解答集

この解答集では、課題集の問題に対する解答と解説を提供します。

⚠️ **注意**: まず自分で解いてから、解答を確認するようにしてください。

---
## 解答1: POSTとGETの違い

### 問題
POSTとGETの違いを説明し、それぞれの使いどころを挙げてください。

### 解答

In [ ]:
<?php

$differences = [
    'データの場所' => [
        'POST' => 'リクエストボディ内に含まれる',
        'GET' => 'URLパラメータとして含まれる（?key=value&...）'
    ],
    'データ容量' => [
        'POST' => '大容量可（php.iniのpost_max_sizeまで）',
        'GET' => '制限あり（約2000文字、ブラウザにより異なる）'
    ],
    'セキュリティ' => [
        'POST' => '高い（履歴に残らない、URLに表示されない）',
        'GET' => '低い（URLに表示される、履歴に残る）'
    ],
    'ブックマーク' => [
        'POST' => '不可（結果ページをブックマークできない）',
        'GET' => '可能（検索結果などをブックマークできる）'
    ]
];

foreach ($differences as $key => $value) {
    echo "{$key}:\n";
    echo "  POST: {$value['POST']}\n";
    echo "  GET: {$value['GET']}\n\n";
}

// POSTが適している例
$postExamples = [
    'お問い合わせフォーム（個人情報を含む）',
    'ログインフォーム（パスワードを含む）',
    '会員登録フォーム',
    'ファイルアップロード',
    '注文フォーム（クレジットカード情報を含む）'
];

// GETが適している例
$getExamples = [
    '検索フォーム（ブックマーク可能）',
    'ページネーション（?page=2）',
    'フィルタリング（?category=php&sort=new）',
    'APIのデータ取得リクエスト'
];

echo "POSTが適している例:\n";
foreach ($postExamples as $example) {
    echo "  - {$example}\n";
}

echo "\nGETが適している例:\n";
foreach ($getExamples as $example) {
    echo "  - {$example}\n";
}

?>

### 解説

**POST vs GET の選び方**:

1. **データを変更する処理** → POST
   - データの登録・更新・削除
   - センシティブな情報の送信

2. **データを取得する処理** → GET
   - 検索・フィルタリング
   - ページネーション
   - ブックマーク可能にしたい場合

**HTTPメソッドの原則（RESTful）**:
- GET: リソースの取得
- POST: リソースの作成
- PUT/PATCH: リソースの更新
- DELETE: リソースの削除

---
## 解答2: フォームデータの受け取り

### 問題
以下のフォームから送信されたデータを安全に受け取る処理を実装してください。

### 解答

In [ ]:
<?php

/**
 * POSTデータを安全に取得する
 * @param string $key フィールド名
 * @param string $default デフォルト値
 * @return string サニタイズ済みの値
 */
function getPost(string $key, string $default = ''): string {
    $value = $_POST[$key] ?? $default;
    return trim($value);
}

// POSTメソッドのチェック
if ($_SERVER['REQUEST_METHOD'] === 'POST') {
    // 各フィールドの値を安全に取得
    $name = getPost('name', '');
    $email = getPost('email', '');
    $message = getPost('message', '');
    
    // 受け取ったデータを表示（サニタイズ済み）
    echo "受け取ったデータ:\n";
    echo "お名前: " . htmlspecialchars($name, ENT_QUOTES, 'UTF-8') . "\n";
    echo "メール: " . htmlspecialchars($email, ENT_QUOTES, 'UTF-8') . "\n";
    echo "内容: " . htmlspecialchars($message, ENT_QUOTES, 'UTF-8') . "\n";
} else {
    echo "このページに直接アクセスしないでください。フォームから送信してください。\n";
}

// 模擬データでテスト
$_POST = [
    'name' => '山田太郎',
    'email' => 'test@example.com',
    'message' => 'お問い合わせです'
];
$_SERVER['REQUEST_METHOD'] = 'POST';

echo "\n--- テストデータ ---\n";
if ($_SERVER['REQUEST_METHOD'] === 'POST') {
    $name = getPost('name', '');
    $email = getPost('email', '');
    $message = getPost('message', '');
    
    echo "お名前: " . htmlspecialchars($name, ENT_QUOTES, 'UTF-8') . "\n";
    echo "メール: " . htmlspecialchars($email, ENT_QUOTES, 'UTF-8') . "\n";
    echo "内容: " . htmlspecialchars($message, ENT_QUOTES, 'UTF-8') . "\n";
}

?>

### 解説

**ポイント**:

1. **`$_SERVER['REQUEST_METHOD']`** でリクエストメソッドを確認

2. **Null合体演算子 `??`** を使用して、キーが存在しない場合のデフォルト値を設定
   - `$_POST['key'] ?? 'default'` の形式

3. **`trim()`** で前後の空白を除去

4. **`htmlspecialchars()`** で出力時にサニタイズ
   - 第2引数: `ENT_QUOTES`（シングルクォートもエスケープ）
   - 第3引数: `'UTF-8'`（文字エンコーディング指定）

---
## 解答3: 入力バリデーション

### 問題
お問い合わせフォーム用のバリデーション関数を作成してください。

### 解答

In [ ]:
<?php

/**
 * お問い合わせフォームのバリデーション
 * @param array $data フォームデータ
 * @return array エラーメッセージの連想配列
 */
function validateContactForm(array $data): array {
    $errors = [];
    
    // お名前：必須、1〜50文字
    if (empty($data['name'])) {
        $errors['name'] = 'お名前は必須です';
    } elseif (mb_strlen($data['name']) < 1) {
        $errors['name'] = 'お名前を入力してください';
    } elseif (mb_strlen($data['name']) > 50) {
        $errors['name'] = 'お名前は50文字以内で入力してください';
    }
    
    // メールアドレス：必須、正しい形式
    if (empty($data['email'])) {
        $errors['email'] = 'メールアドレスは必須です';
    } elseif (!filter_var($data['email'], FILTER_VALIDATE_EMAIL)) {
        $errors['email'] = '正しいメールアドレスを入力してください';
    }
    
    // 電話番号：任意、半角数字とハイフンのみ
    if (!empty($data['phone'])) {
        if (!preg_match('/^[0-9\-]+$/', $data['phone'])) {
            $errors['phone'] = '電話番号は半角数字とハイフンのみ入力可能です';
        }
    }
    
    // お問い合わせ内容：必須、10〜1000文字
    if (empty($data['message'])) {
        $errors['message'] = 'お問い合わせ内容は必須です';
    } elseif (mb_strlen($data['message']) < 10) {
        $errors['message'] = 'お問い合わせ内容は10文字以上で入力してください';
    } elseif (mb_strlen($data['message']) > 1000) {
        $errors['message'] = 'お問い合わせ内容は1000文字以内で入力してください';
    }
    
    return $errors;
}

// テストケース
$testCases = [
    '正しいデータ' => [
        'name' => '山田太郎',
        'email' => 'test@example.com',
        'phone' => '090-1234-5678',
        'message' => 'お問い合わせ内容です。10文字以上あります。'
    ],
    '名前が空' => [
        'name' => '',
        'email' => 'test@example.com',
        'phone' => '090-1234-5678',
        'message' => 'お問い合わせ内容です。10文字以上あります。'
    ],
    '無効なメール' => [
        'name' => '山田太郎',
        'email' => 'invalid-email',
        'phone' => '090-1234-5678',
        'message' => 'お問い合わせ内容です。10文字以上あります。'
    ],
    '内容が短い' => [
        'name' => '山田太郎',
        'email' => 'test@example.com',
        'phone' => '090-1234-5678',
        'message' => '短い'
    ],
    '無効な電話番号' => [
        'name' => '山田太郎',
        'email' => 'test@example.com',
        'phone' => 'あいうえお',
        'message' => 'お問い合わせ内容です。10文字以上あります。'
    ]
];

foreach ($testCases as $caseName => $data) {
    echo "\n【{$caseName}】\n";
    $errors = validateContactForm($data);
    if (empty($errors)) {
        echo "  ✓ パス\n";
    } else {
        foreach ($errors as $field => $error) {
            echo "  ✗ {$field}: {$error}\n";
        }
    }
}

?>

### 解説

**バリデーションの重要ポイント**:

1. **`empty()`** で値が空かチェック
   - 空文字、null、false、0などが空と判定される

2. **`mb_strlen()`** でマルチバイト文字の長さを取得
   - 日本語などの文字数を正しくカウント

3. **`filter_var()`** でメールアドレスの形式チェック
   - `FILTER_VALIDATE_EMAIL` フラグを使用

4. **`preg_match()`** で正規表現によるパターンチェック
   - `/^[0-9\-]+$/` は数字とハイフンのみにマッチ
   - `^` は文字列の先頭、`$` は文字列の末尾

**実務のヒント**:
- 任意項目は `!empty()` で値が存在する場合のみチェック
- エラーメッセージは具体的に（どうすればいいか示す）

---
## 解答4: サニタイズ（XSS対策）

### 問題
ユーザー入力を安全にHTML出力するためのサニタイズ関数を作成してください。

### 解答

In [ ]:
<?php

/**
 * 文字列をサニタイズする
 * @param string $value サニタイズ対象の文字列
 * @return string サニタイズ後の文字列
 */
function sanitize(string $value): string {
    return htmlspecialchars($value, ENT_QUOTES, 'UTF-8');
}

/**
 * 配列を再帰的にサニタイズする
 * @param array $data サニタイズ対象の配列
 * @return array サニタイズ後の配列
 */
function sanitizeArray(array $data): array {
    return array_map(function($value) {
        if (is_array($value)) {
            return sanitizeArray($value);
        }
        return htmlspecialchars($value, ENT_QUOTES, 'UTF-8');
    }, $data);
}

// テスト：悪意のある入力
$maliciousInputs = [
    '<script>alert("XSS")</script>',
    '<img src="x" onerror="alert(1)">',
    '<a href="javascript:alert(1)">クリック</a>',
    '正常な<br>テキスト'
];

echo "=== 文字列のサニタイズ ===\n";
foreach ($maliciousInputs as $input) {
    echo "\n元の値: {$input}\n";
    echo "安全な値: " . sanitize($input) . "\n";
}

// 配列のサニタイズ
$maliciousArray = [
    'name' => '<script>alert(1)</script>',
    'email' => 'test@example.com',
    'comment' => '<img src=x onerror=alert(1)>',
    'nested' => [
        'title' => '<b>太字</b>',
        'desc' => '説明<script>alert(2)</script>'
    ]
];

echo "\n=== 配列のサニタイズ ===\n";
$safeArray = sanitizeArray($maliciousArray);
print_r($safeArray);

// 出力時の使用例
echo "\n=== 安全なHTML出力の例 ===\n";
$userInput = '<script>alert("XSS")</script>';
echo '<div>' . sanitize($userInput) . '</div>' . "\n";

?>

### 解説

**XSS（クロスサイトスクリプティング）対策**:

1. **`htmlspecialchars()`** の役割
   - 特殊文字をHTMLエンティティに変換
   - `<` → `&lt;`, `>` → `&gt;`
   - `"` → `&quot;`, `'` → `&#039;`
   - `&` → `&amp;`

2. **`ENT_QUOTES`** フラグ
   - シングルクォートもエスケープ
   - セキュリティ上、必ず指定することを推奨

3. **`'UTF-8'`** エンコーディング指定
   - 文字エンコーディングを明示的に指定
   - 文字化けやセキュリティ問題を防止

**XSS攻撃の種類**:
- **Reflected XSS**: URLパラメータなどで実行
- **Stored XSS**: データベースに保存され、表示時に実行
- **DOM-based XSS**: JavaScriptのDOM操作で実行

**対策の基本**:
- 入力値を信頼しない
- 出力時に必ずサニタイズ
- Content Security Policy (CSP) の使用を検討

---
## 解答5: CSRFトークンの実装

### 問題
CSRF対策のためのトークン生成・検証関数を作成してください。

### 解答

In [ ]:
<?php

// セッション開始（通常はファイルの先頭で呼び出し）
if (session_status() === PHP_SESSION_NONE) {
    session_start();
}

/**
 * CSRFトークンを生成する
 * @return string トークン
 */
function generateCsrfToken(): string {
    // まだトークンがない場合は生成
    if (!isset($_SESSION['csrf_token'])) {
        $_SESSION['csrf_token'] = bin2hex(random_bytes(32));
    }
    return $_SESSION['csrf_token'];
}

/**
 * CSRFトークンを検証する
 * @param string $token 検証するトークン
 * @return bool 有効ならtrue
 */
function validateCsrfToken(string $token): bool {
    return isset($_SESSION['csrf_token']) && hash_equals($_SESSION['csrf_token'], $token);
}

/**
 * フォーム用のhiddenフィールドHTMLを生成
 * @return string HTMLタグ
 */
function getCsrfField(): string {
    $token = generateCsrfToken();
    return '<input type="hidden" name="csrf_token" value="' . htmlspecialchars($token, ENT_QUOTES, 'UTF-8') . '">';
}

/**
 * CSRFトークンを再生成する（重要な操作の後など）
 */
function regenerateCsrfToken(): void {
    $_SESSION['csrf_token'] = bin2hex(random_bytes(32));
}

// テスト
echo "=== CSRFトークンテスト ===\n\n";

// トークン生成
$token1 = generateCsrfToken();
echo "生成したトークン: {$token1}\n";
echo "トークン長: " . strlen($token1) . "文字（64文字 = 32バイト）\n";

// 2回目は同じトークンが返ることを確認
$token2 = generateCsrfToken();
echo "\n2回目のトークン: {$token2}\n";
echo "同じトークン: " . ($token1 === $token2 ? 'Yes' : 'No') . "\n";

// 検証テスト
echo "\n--- 検証テスト ---\n";
echo "有効なトークンの検証: " . (validateCsrfToken($token1) ? '✓ Pass' : '✗ Fail') . "\n";
echo "無効なトークンの検証: " . (validateCsrfToken('invalid_token') ? '✗ Fail' : '✓ Pass') . "\n";
echo "空文字の検証: " . (validateCsrfToken('') ? '✗ Fail' : '✓ Pass') . "\n";

// hiddenフィールド生成
echo "\n--- HTML出力 ---\n";
echo getCsrfField() . "\n";

// トークン再生成
echo "\n--- トークン再生成 ---\n";
$oldToken = $token1;
regenerateCsrfToken();
$newToken = generateCsrfToken();
echo "旧トークン: {$oldToken}\n";
echo "新トークン: {$newToken}\n";
echo "異なるトークン: " . ($oldToken !== $newToken ? 'Yes' : 'No') . "\n";

?>

### 解説

**CSRF（クロスサイトリクエストフォージェリ）対策**:

1. **`random_bytes(32)`** で暗号的に安全な乱数を生成
   - 32バイト（256ビット）のランダム値
   - `bin2hex()` で16進数の文字列に変換

2. **`hash_equals()`** でタイミング攻撃を防ぐ
   - 文字列比較に一定時間をかける
   - 通常の `===` 比較は時間差から情報が漏れる可能性

3. **セッションに保存**
   - ユーザーごとに一意のトークン
   - リクエスト間で同じトークンを使用

**CSRF攻撃の流れ**:

```
1. ユーザーが銀行サイトにログイン
2. 攻撃者が作成したページを閲覧
3. ページ内から銀行サイトへリクエスト送信
4. ユーザーのクッキーが送信され、不正な処理実行
```

**トークンによる対策**:
- フォームにランダムなトークンを埋め込む
- 送信時にトークンを検証
- 攻撃者はトークンを知りえないため不正リクエストを防げる

**実装のポイント**:
- 重要な操作（パスワード変更、送金など）の後はトークンを再生成
- GETリクエストでは状態を変更しない（CSRFの影響を受けにくい）
- SameSite属性のクッキーも併用するとさらに安全

---
## 解答6: ファイルアップロード

### 問題
安全なファイルアップロード処理を実装してください。

### 解答

In [ ]:
<?php

/**
 * アップロードされたファイルを検証する
 * @param array $file $_FILES['xxx']の値
 * @param int $maxSize 最大サイズ（バイト）
 * @return array ['valid' => bool, 'error' => string]
 */
function validateUploadedFile(array $file, int $maxSize = 2097152): array {
    $result = ['valid' => true, 'error' => ''];
    
    // アップロードエラーチェック
    if ($file['error'] !== UPLOAD_ERR_OK) {
        $errorMessages = [
            UPLOAD_ERR_INI_SIZE => 'ファイルサイズがphp.iniのupload_max_filesizeを超えています',
            UPLOAD_ERR_FORM_SIZE => 'ファイルサイズがMAX_FILE_SIZEを超えています',
            UPLOAD_ERR_PARTIAL => 'ファイルが一部しかアップロードされていません',
            UPLOAD_ERR_NO_FILE => 'ファイルが選択されていません',
            UPLOAD_ERR_NO_TMP_DIR => '一時フォルダがありません',
            UPLOAD_ERR_CANT_WRITE => 'ファイルの書き込みに失敗しました',
            UPLOAD_ERR_EXTENSION => 'PHP拡張機能がファイルのアップロードを停止しました'
        ];
        $result['valid'] = false;
        $result['error'] = $errorMessages[$file['error']] ?? '不明なエラーが発生しました';
        return $result;
    }
    
    // ファイルサイズチェック
    if ($file['size'] > $maxSize) {
        $result['valid'] = false;
        $result['error'] = "ファイルサイズは" . round($maxSize / 1024 / 1024, 1) . "MB以下にしてください";
        return $result;
    }
    
    // 許可するMIMEタイプ
    $allowedMimeTypes = [
        'image/jpeg',
        'image/png',
        'image/gif',
        'image/webp'
    ];
    
    // ファイルタイプチェック（MIMEタイプ）
    $finfo = finfo_open(FILEINFO_MIME_TYPE);
    $mimeType = finfo_file($finfo, $file['tmp_name']);
    finfo_close($finfo);
    
    if (!in_array($mimeType, $allowedMimeTypes)) {
        $result['valid'] = false;
        $result['error'] = '画像ファイル（JPEG, PNG, GIF, WebP）のみアップロードできます';
        return $result;
    }
    
    // 拡張子チェック
    $allowedExtensions = ['jpg', 'jpeg', 'png', 'gif', 'webp'];
    $extension = strtolower(pathinfo($file['name'], PATHINFO_EXTENSION));
    
    if (!in_array($extension, $allowedExtensions)) {
        $result['valid'] = false;
        $result['error'] = '許可されていない拡張子です';
        return $result;
    }
    
    // MIMEタイプと拡張子の整合性チェック
    $mimeToExtension = [
        'image/jpeg' => ['jpg', 'jpeg'],
        'image/png' => ['png'],
        'image/gif' => ['gif'],
        'image/webp' => ['webp']
    ];
    
    if (!isset($mimeToExtension[$mimeType]) || !in_array($extension, $mimeToExtension[$mimeType])) {
        $result['valid'] = false;
        $result['error'] = 'ファイルタイプと拡張子が一致しません';
        return $result;
    }
    
    return $result;
}

/**
 * 安全なファイル名を生成する
 * @param string $filename 元のファイル名
 * @return string 安全なファイル名
 */
function generateSafeFilename(string $filename): string {
    // 拡張子を取得
    $extension = strtolower(pathinfo($filename, PATHINFO_EXTENSION));
    
    // ファイル名から拡張子を除いた部分を取得
    $basename = pathinfo($filename, PATHINFO_FILENAME);
    
    // 特殊文字をアンダースコアに置換
    $safeBasename = preg_replace('/[^A-Za-z0-9_\-\p{Hiragana}\p{Katakana}\p{Han}]/u', '_', $basename);
    
    // タイムスタンプを付けて重複回避
    $timestamp = time();
    
    return "{$timestamp}_{$safeBasename}.{$extension}";
}

// テスト用の模擬ファイルデータ
$testFiles = [
    '有効な画像' => [
        'name' => 'photo.jpg',
        'type' => 'image/jpeg',
        'size' => 1024000,
        'error' => UPLOAD_ERR_OK,
        'tmp_name' => '/tmp/test'  // 模擬用
    ],
    '大きすぎるファイル' => [
        'name' => 'large.jpg',
        'type' => 'image/jpeg',
        'size' => 5000000,
        'error' => UPLOAD_ERR_OK,
        'tmp_name' => '/tmp/test'
    ],
    '無効な形式' => [
        'name' => 'script.php',
        'type' => 'application/x-httpd-php',
        'size' => 1000,
        'error' => UPLOAD_ERR_OK,
        'tmp_name' => '/tmp/test'
    ],
    'アップロードエラー' => [
        'name' => 'photo.jpg',
        'type' => 'image/jpeg',
        'size' => 1000,
        'error' => UPLOAD_ERR_NO_FILE,
        'tmp_name' => '/tmp/test'
    ]
];

echo "=== ファイルアップロード検証テスト ===\n\n";

// 注意: このテストではMIMEタイプチェックをスキップ（模擬データのため）
foreach ($testFiles as $caseName => $file) {
    echo "【{$caseName}】\n";
    
    // エラーチェックは正常に動作
    if ($file['error'] !== UPLOAD_ERR_OK) {
        $result = validateUploadedFile($file);
        echo "  ✗ {$result['error']}\n";
    } elseif ($file['size'] > 2097152) {
        $result = validateUploadedFile($file);
        echo "  ✗ {$result['error']}\n";
    } else {
        echo "  ✓ 有効なファイル\n";
        echo "  安全なファイル名: " . generateSafeFilename($file['name']) . "\n";
    }
    echo "\n";
}

// ファイル名生成のテスト
echo "=== 安全なファイル名生成 ===\n";
$testFilenames = [
    'normal.jpg',
    '日本語のファイル名.png',
    '../../../etc/passwd.jpg',
    'script.php.jpg',
    'file with spaces.gif'
];

foreach ($testFilenames as $filename) {
    echo "{$filename} → " . generateSafeFilename($filename) . "\n";
}

?>

### 解説

**ファイルアップロードのセキュリティリスク**:

| 脅威 | 説明 | 対策 |
|------|------|------|
| スクリプト実行 | PHPファイルをアップロードされ実行される | 拡張子チェック、保存ディレクトリの実行禁止 |
| ディレクトリトラバーサル | `../../../etc/passwd` などのパス | ファイル名のサニタイズ |
| 大容量DoS | 巨大なファイルアップロード | サイズ制限 |
| 偽装 | 拡張子と実体が異なる | MIMEタイプチェック、魔術バイト検証 |

**実装のポイント**:

1. **アップロードディレクトリは公開しない
   - `webroot/uploads/` ではなく `/var/uploads/` に保存
   - PHPスクリプトを実行できないよう `.htaccess` で設定

2. **ファイル名のサニタイズ
   - ユーザー入力のファイル名をそのまま使わない
   - タイムスタンプ + ランダム値で一意な名前にする

3. **MIMEタイプ検証
   - `$_FILES['type']` はユーザー入力で偽装可能
   - `finfo_open()` で実際のファイルタイプを確認

4. **保存後のアクセス制御
   - ダウンロード用スクリプトを通して配布
   - 直接アクセスできない場所に保存

---
## 解答7: 実践問題 - アンケートフォーム

### 問題
以下の仕様を持つアンケートフォームのバリデーション関数を作成してください。

### 解答

In [ ]:
<?php

/**
 * アンケートフォームのバリデーション
 * @param array $data フォームデータ
 * @return array ['valid' => bool, 'errors' => array, 'data' => array]
 */
function validateSurveyForm(array $data): array {
    $result = [
        'valid' => true,
        'errors' => [],
        'data' => []
    ];
    
    // 満足度（1〜5の必須選択）
    if (empty($data['satisfaction'])) {
        $result['errors']['satisfaction'] = '満足度を選択してください';
        $result['valid'] = false;
    } elseif (!in_array($data['satisfaction'], ['1', '2', '3', '4', '5'], true)) {
        $result['errors']['satisfaction'] = '満足度は1〜5で選択してください';
        $result['valid'] = false;
    } else {
        $result['data']['satisfaction'] = (int)$data['satisfaction'];
    }
    
    // 年齢層（必須選択）
    $validAgeGroups = ['under20', '20s', '30s', '40s', '50s', '60s', 'over70'];
    if (empty($data['age_group'])) {
        $result['errors']['age_group'] = '年齢層を選択してください';
        $result['valid'] = false;
    } elseif (!in_array($data['age_group'], $validAgeGroups)) {
        $result['errors']['age_group'] = '無効な年齢層です';
        $result['valid'] = false;
    } else {
        $result['data']['age_group'] = $data['age_group'];
    }
    
    // 改善点（任意、最大500文字）
    if (!empty($data['improvements'])) {
        $improvements = trim($data['improvements']);
        if (mb_strlen($improvements) > 500) {
            $result['errors']['improvements'] = '改善点は500文字以内で入力してください';
            $result['valid'] = false;
        } else {
            $result['data']['improvements'] = $improvements;
        }
    }
    
    // 再利用意向（必須選択）
    if (empty($data['would_recommend']) || !is_array($data['would_recommend'])) {
        $result['errors']['would_recommend'] = '再利用意向を選択してください';
        $result['valid'] = false;
    } else {
        $result['data']['would_recommend'] = $data['would_recommend'];
    }
    
    // メールアドレス（任意、正しい形式の場合のみ保存）
    if (!empty($data['email'])) {
        if (filter_var($data['email'], FILTER_VALIDATE_EMAIL)) {
            $result['data']['email'] = trim($data['email']);
        } else {
            $result['errors']['email'] = '正しいメールアドレスを入力してください';
            $result['valid'] = false;
        }
    }
    
    return $result;
}

// テストケース
$testCases = [
    '全て正しいデータ' => [
        'satisfaction' => '5',
        'age_group' => '30s',
        'improvements' => '非常に良いでした。',
        'would_recommend' => ['yes']
    ],
    '満足度未選択' => [
        'satisfaction' => '',
        'age_group' => '30s',
        'improvements' => '',
        'would_recommend' => ['yes']
    ],
    '無効な満足度' => [
        'satisfaction' => '10',
        'age_group' => '30s',
        'improvements' => '',
        'would_recommend' => ['yes']
    ],
    '改善点が長すぎる' => [
        'satisfaction' => '3',
        'age_group' => '20s',
        'improvements' => str_repeat('あ', 501),
        'would_recommend' => ['yes']
    ],
    '無効なメール' => [
        'satisfaction' => '4',
        'age_group' => '40s',
        'improvements' => '',
        'would_recommend' => ['no'],
        'email' => 'invalid-email'
    ],
    '有効なメール付き' => [
        'satisfaction' => '5',
        'age_group' => '50s',
        'improvements' => '',
        'would_recommend' => ['yes'],
        'email' => 'test@example.com'
    ]
];

echo "=== アンケートフォーム検証テスト ===\n\n";

foreach ($testCases as $caseName => $data) {
    echo "【{$caseName}】\n";
    $result = validateSurveyForm($data);
    
    if ($result['valid']) {
        echo "  ✓ バリデーションパス\n";
        echo "  データ: " . json_encode($result['data'], JSON_UNESCAPED_UNICODE) . "\n";
    } else {
        echo "  ✗ バリデーションエラー:\n";
        foreach ($result['errors'] as $field => $error) {
            echo "    - {$field}: {$error}\n";
        }
    }
    echo "\n";
}

?>

### 解説

**アンケートフォームの実装ポイント**:

1. **ラジオボタン（満足度）**
   - 単一選択なので文字列または数値で受け取る
   - `in_array()` で有効な値かチェック
   - 第3引数 `true` で厳密な型比較

2. **セレクトボックス（年齢層）**
   - 有効な選択肢を配列で定義
   - 配列に含まれるかチェック

3. **テキストエリア（改善点）**
   - 任意項目は `!empty()` で値がある場合のみチェック
   - 最大文字数を設定

4. **チェックボックス（再利用意向）**
   - 配列として受け取る
   - `is_array()` で型チェック

5. **メールアドレス（任意）**
   - 値がある場合のみバリデーション
   - 無効な場合はエラーに追加

**HTMLフォームの例**:

```html
<!-- 満足度（ラジオボタン） -->
<input type="radio" name="satisfaction" value="1" required>
<input type="radio" name="satisfaction" value="2">
...

<!-- 年齢層（セレクトボックス） -->
<select name="age_group" required>
  <option value="">選択してください</option>
  <option value="20s">20代</option>
  ...
</select>

<!-- 再利用意向（チェックボックス） -->
<input type="checkbox" name="would_recommend[]" value="yes" required>
```

---
## まとめ

### 学習内容の振り返り

| トピック | 重要ポイント | 関数/設定 |
|---------|-------------|-----------|
| POST vs GET | データの場所、容量、セキュリティ | `$_SERVER['REQUEST_METHOD']` |
| フォーム受取 | デフォルト値、NULL合体演算子 | `$_POST['key'] ?? 'default'` |
| バリデーション | 必須チェック、形式チェック | `filter_var()`, `mb_strlen()` |
| XSS対策 | 特殊文字のエスケープ | `htmlspecialchars()` |
| CSRF対策 | トークン生成と検証 | `random_bytes()`, `hash_equals()` |
| ファイルアップロード | タイプチェック、サイズ制限 | `finfo_open()`, `$_FILES` |

### セキュリティ実装チェックリスト

```
☐ 全てのユーザー入力をバリデートしている
☐ HTML出力時にサニタイズしている
☐ CSRFトークンを使用している
☐ ファイルアップロードの検証をしている
☐ エラーメッセージで詳細な情報を出しすぎていない
☐ パスワードはハッシュ化して保存
☐ HTTPSを使用している
```

### 次のステップ

- 基礎の復習（第8章）でこれまでの内容を定着させましょう
- 実際のフォームを作成して理解を深めましょう